In [1]:
import os, re
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [21]:
################################################################################
inFilePath = './outputs/part09.log'
with open(inFilePath, 'r') as f:
    raw = pd.Series(f.read().split('\n')) # read file, each row = line

FileNotFoundError: [Errno 2] No such file or directory: './outputs/part09.log'

Get series of diminishing losses across epochs by using regular expression to parse through the log and extracting Loss numbers on test data set

In [77]:
lossRE = re.compile(r'\(loss: (\d+\.\d+).+') # RE to capture loss value as text
tmp = raw.str.extract(lossRE)[0] # extract loss values as string
loss = tmp.dropna().values.astype(np.float) # drop NA, convert string to flaot

In [ ]:
plt.plot(range(1,31), loss)
plt.title('Loss Across Training Epochs')
plt.xlabel('Training Epochs')
plt.ylabel('Loss Value')
plt.show()

In [103]:
np.sum(raw.str.find('TEST : total')>=0)

17

In [ ]:
def getBestTestPerf(raw):
    found = raw.str.find('TEST : total') >= 0
    

In [19]:
ts = re.compile(r'TEST.+(\w\-\w+).+?(\d\.\d+).+?(\d\.\d+).+?(\d\.\d+).+(\d\.\d+)')
def getTestPerf(line):
    m = ts.match(line)
    # return tuple with: (entity, F1, recall, precision)
    return (m[1], float(m[2]), float(m[3]), float(m[4]))

In [23]:
getTestPerf(s)

('I-ORG', 0.0221, 0.012, 0.1389)

In [22]:
s = "TEST : I-ORG : test_f1: 0.0221 test_rec: 0.0120 test_pre: 0.1389 test_acc: 0.0000 | {'O': 633, 'I-PER': 143, 'B-PER': 26, 'B-LOC': 22, 'I-LOC': 1}"
